# **Security Course**
### <span style="font-weight:bold;">Instructor:</span> <span style="font-size:19px;"><a href="https://ir.linkedin.com/in/hrshahriari">Dr. Hamid Reza Shahriari</a></span>
### <span style="font-weight:bold;">Organization</span>
### <span style="font-size:19px;"><a href="https://aut.ac.ir/en">Amirkabir University of Technology (Tehran Polytechnic)</a></span>
<img src="../../assets/AKUT-white-logo.png" alt="Amirkabir University of Technology logo" style="width:300px;">

# **Host Certificate Checker**
In this notebook, we will walk through the steps of implementing a code to show details of sertificates of the input domain.

### Imports

In [1]:
import socket
import ssl
import datetime

### Read Details of Certificate

In [2]:
def read_details(cert):
    version = cert['version']
    serial_number = cert['serialNumber']
    subject = dict(x[0] for x in cert['subject'])
    issued_to = subject.get("commonName", "N/A")
    issuer = dict(x[0] for x in cert['issuer'])
    issued_by = issuer.get("commonName", "N/A")
    valid_from = datetime.datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y %Z")
    valid_to = datetime.datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y %Z")
    days_remaining = (valid_to - datetime.datetime.utcnow()).days

    return version, serial_number, issued_to, issued_by, valid_from, valid_to, days_remaining

### Show Details of Certificate

In [3]:
def show_details(hostname, version, serial_number, issued_to, issued_by, valid_from, valid_to, days_remaining):
    print(f"Certificate for {hostname}")
    print(f"  Version: {version}")
    print(f"  Serial Number: {serial_number}")
    print(f"  Issued To: {issued_to}")
    print(f"  Issued By: {issued_by}")
    print(f"  Valid From: {valid_from}")
    print(f"  Valid To: {valid_to}")
    print(f"  Days Remaining: {days_remaining}")
    print()
    if days_remaining > 0:
        print("Status: Certificate is valid.")
    else:
        print("Status: Certificate has expired.")

### Certificate Checker

In [4]:
def check_certificate(hostname, port=443):
    context = ssl.create_default_context()
    conn = context.wrap_socket(
        socket.socket(socket.AF_INET),
        server_hostname=hostname,
    )
    try:
        conn.connect((hostname, port))
        cert = conn.getpeercert()
        version, serial_number, issued_to, issued_by, valid_from, valid_to, days_remaining= read_details(cert)
        show_details(hostname, version, serial_number, issued_to, issued_by, valid_from, valid_to, days_remaining)
    except Exception as e:
        print(f"Failed to retrieve certificate for {hostname}. Error: {e}")
    finally:
        conn.close()

### Testing Implementated certificated checker

In [5]:
hostname = input("Enter the hostname (e.g., example.com): ")
input_port = input("Enter the port (e.g., 443): ")
port = 443 if input_port.strip() == '' else int(input_port)

check_certificate(hostname, port)

Certificate for google.com
  Version: 3
  Serial Number: 41A81582740B7A9009B461E71720C0F4
  Issued To: *.google.com
  Issued By: WR2
  Valid From: 2024-10-21 08:36:57
  Valid To: 2025-01-13 08:36:56
  Days Remaining: 42

Status: Certificate is valid.
